In [143]:
# taken from https://towardsdatascience.com/perceptron-algorithm-in-python-f3ac89d2e537

In [144]:
# imports

from sklearn import datasets
import seaborn as sns
import pandas as pd
import numpy as np

In [145]:
class Generator:

    def __init__ (self):
        self.columns = ['x', 'y', 'labels']

    def makeDF(self):
        return pd.DataFrame(self.makeData(), columns=self.columns)
        
    def makeData(self):
        _x, _y = datasets.make_blobs(
            n_samples = number_of_points,
            n_features = 2,
            centers = 2,
            cluster_std = 1.05,
            random_state = 2)
        return list(zip(_x[:,0], _x[:,1], _y))

In [146]:
class Plotter:

    def plot(self, df):
        sns.scatterplot(
            x = df.columns[0], 
            y = df.columns[1], 
            data = df,
            hue = df.columns[2])

In [147]:
class RandomData: 

    def __init__(self, number_of_points):
        self.df = Generator().makeDF(number_of_points)
        self.plotter = Plotter()

    def plot(self):
        self.plotter.plot(self.generator.df)

In [148]:
class Perceptron:

    def __init__(self, data):
        self.epochs = 100
        self.learning_rate = 0.1
        self.data = data
        self.labels = self.data.iloc[:,-1]
        self.samples = self.data.iloc[:,:-1]
        self.sample_count = self.samples.shape[0]
        self.feature_count  = self.samples.shape[1]
        self.theta = np.zeros((self.feature_count + 1, 1))
        self.activation_function = ActivationFunction(0.0)
        self.miscalssified_counts = []

    def train():
        for _ in range(epochs):
            for _i, _sample_i in enumerate(self.samples):
                self.misclassified_counts[_i] = 0

    def neuron(index, sample):
        _prediction = self.predict(sample)
        self.evaluate_prediction(index, sample, _prediction)
    
    def predict(sample):
        _processed_sample = np.insert(sample, 0, 1).reshape(-1,1)
        _dot_product = np.dot(_processed_sample.T, _theta)
        _prediction = self.activation_function.check(_dot_product)
        return _prediction
    
    def evaluate_prediction(index, sample, prediction):
        if np.squeeze(prediction, self.labels[index]) == 0:
            return
        self.update_theta(index, sample, prediction)
        self.increment_misclassification_count(index)

    def update_theta(index, sample, prediction):
        self.theta += self.learning_rate*sample*(self.labels(index) - prediction)

    def increment_misclassification_count(index):
        self.misclassified_count[index] += 1

    def print_initialization(self):
        print(f'epochs: {self.epochs}')
        print(f'learning rate: {self.learning_rate}')
        print(f'sample count: {self.sample_count}')
        print(f'feature count: {self.feature_count}\n')
        print(f'samples:\n{self.samples.head()}\n')
        print(f'labels:\n{self.labels.head()}\n')
        print(f'theta:\n{self.theta}\n')


In [149]:
class ActivationFunction:

    def __init__(self, threashold):
        self.threashold = threashold

    def check(self, data):
        return [activate(_x) if _x > self.threashold else deactivate(_x) for _x in data]

    def activate(self, data):
        return 1.0

    def deactivate(self, data):
        return 0.0

In [150]:
_data = RandomData(250)
_perceptron = Perceptron(_data.df)
_perceptron.epochs = 10
_perceptron.print_initialization()

TypeError: __init__() missing 1 required positional argument: 'number_of_points'